In [1]:
import my_utils
import pandas as pd
import numpy as np
import pickle
import pm4py
from pm4py.objects.log.util.log import project_traces
from pm4py.objects.log.util import interval_lifecycle
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features
from pm4py.objects.log.obj import EventLog, Trace

import warnings
warnings.filterwarnings("ignore")

In [2]:
# function to project the trace
def project_nth(log, index):
    print(str(project_traces(log)[index]))

In [45]:
df = pd.read_csv("data/Sampled_BPIC17_offer_log.csv")
df.head()

,index,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,EventOrigin,OfferID,lifecycle:transition
0,0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,Offer,NaN,complete
1,1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer,Offer_247135719,complete
2,2,Offer_247135719,O_Sent (online only),User_17,2016/01/02 18:19:21.330,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_440662877,Offer,Offer_247135719,complete
3,3,Offer_247135719,O_Cancelled,User_17,2016/01/02 18:21:26.034,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_591416028,Offer,Offer_247135719,complete
4,4,Offer_941964966,O_Create Offer,User_17,2016/01/02 18:21:42.022,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer_941964966,Offer,NaN,complete


In [79]:
def drop_columns(df):
    for col in df.columns:
        if df[col].nunique() == 1:
            print(col)
            df.drop(columns=col, inplace=True)

In [80]:
drop_columns(df)
df

,index,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,(case) Accepted,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,OfferID
0,0,Offer_247135719,O_Create Offer,User_17,2016/01/02 18:17:05.720,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,Created,Offer_247135719,NaN
1,1,Offer_247135719,O_Created,User_17,2016/01/02 18:17:08.762,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_124849367,Offer_247135719
2,2,Offer_247135719,O_Sent (online only),User_17,2016/01/02 18:19:21.330,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_440662877,Offer_247135719
3,3,Offer_247135719,O_Cancelled,User_17,2016/01/02 18:21:26.034,Variant 8,8,True,Application_196483749,0,10000.0,201.76,57,10000.0,False,statechange,OfferState_591416028,Offer_247135719
4,4,Offer_941964966,O_Create Offer,User_17,2016/01/02 18:21:42.022,Variant 1,1,True,Application_196483749,0,4100.0,201.76,57,10000.0,False,Created,Offer_941964966,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016/02/18 22:50:01.446,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,Created,Offer_1064426652,NaN
22405,22405,Offer_1064426652,O_Created,User_23,2016/02/18 22:50:02.655,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_189916093,Offer_1064426652
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016/02/18 22:50:22.174,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_24747308,Offer_1064426652
22407,22407,Offer_1064426652,O_Returned,User_117,2016/02/29 19:08:56.200,Variant 2,2,True,Application_1574359134,767,6000.0,118.56,59,6000.0,True,statechange,OfferState_1809485991,Offer_1064426652


In [81]:
input = {
    "Case ID" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Complete Timestamp": "ts",
    "(case) Accepted" : "y"  
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["t"] = my_utils.dt(df)

In [85]:
df['OfferID'] = df['OfferID'].fillna('Unknown')
df['(case) Selected'] = df['(case) Selected'].astype(int)
df['y'] = df['y'].astype(int)

df['year'] = df['ts'].dt.year
df['month'] = df['ts'].dt.month

drop_columns(df)

df.head()

year


,index,caseid,activity,resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,OfferID,t,month
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000.0,0,Created,Offer_247135719,Unknown,0 days 00:00:03.042000,1
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000.0,0,statechange,OfferState_124849367,Offer_247135719,0 days 00:02:12.568000,1
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000.0,0,statechange,OfferState_440662877,Offer_247135719,0 days 00:02:04.704000,1
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000.0,0,statechange,OfferState_591416028,Offer_247135719,0 days 00:02:04.704000,1
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,1,Application_196483749,0,4100.0,201.76,57,10000.0,0,Created,Offer_941964966,Unknown,0 days 00:00:01.551000,1


In [86]:
df.convert_dtypes()

,index,caseid,activity,resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,OfferID,t,month
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000,0,Created,Offer_247135719,Unknown,0 days 00:00:03.042000,1
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000,0,statechange,OfferState_124849367,Offer_247135719,0 days 00:02:12.568000,1
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000,0,statechange,OfferState_440662877,Offer_247135719,0 days 00:02:04.704000,1
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,1,Application_196483749,0,10000.0,201.76,57,10000,0,statechange,OfferState_591416028,Offer_247135719,0 days 00:02:04.704000,1
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,1,Application_196483749,0,4100.0,201.76,57,10000,0,Created,Offer_941964966,Unknown,0 days 00:00:01.551000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016-02-18 22:50:01.446,Variant 2,2,1,Application_1574359134,767,6000.0,118.56,59,6000,1,Created,Offer_1064426652,Unknown,0 days 00:00:01.209000,2
22405,22405,Offer_1064426652,O_Created,User_23,2016-02-18 22:50:02.655,Variant 2,2,1,Application_1574359134,767,6000.0,118.56,59,6000,1,statechange,OfferState_189916093,Offer_1064426652,0 days 00:00:19.519000,2
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016-02-18 22:50:22.174,Variant 2,2,1,Application_1574359134,767,6000.0,118.56,59,6000,1,statechange,OfferState_24747308,Offer_1064426652,10 days 20:18:34.026000,2
22407,22407,Offer_1064426652,O_Returned,User_117,2016-02-29 19:08:56.200,Variant 2,2,1,Application_1574359134,767,6000.0,118.56,59,6000,1,statechange,OfferState_1809485991,Offer_1064426652,0 days 04:34:49.631000,2


In [87]:
a = my_utils.aggregation_encoding(df)
a

,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_26,User_23,User_73,User_125,User_86,User_142,User_76,User_141,User_32,avg_t
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,"[1, 1, 1, 1, 1]","[Application_2131314372, Application_213131437...","[956, 956, 956, 956, 956]","[14500.0, 14500.0, 14500.0, 14500.0, 14500.0]","[200.0, 200.0, 200.0, 200.0, 200.0]","[96, 96, 96, 96, 96]","[16000.0, 16000.0, 16000.0, 16000.0, 16000.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 11:21:59.362200
Offer_1001553250,"[0, 0, 0, 0, 0]","[Application_1607028451, Application_160702845...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[175.0, 175.0, 175.0, 175.0, 175.0]","[127, 127, 127, 127, 127]","[17500.0, 17500.0, 17500.0, 17500.0, 17500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2 days 22:47:28.444400
Offer_1002136393,"[1, 1, 1, 1, 1]","[Application_638926349, Application_638926349,...","[969, 969, 969, 969, 969]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[343.25, 343.25, 343.25, 343.25, 343.25]","[20, 20, 20, 20, 20]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5 days 22:58:29.630800
Offer_1002236598,"[0, 0, 0, 0, 0]","[Application_818693455, Application_818693455,...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[210.0, 210.0, 210.0, 210.0, 210.0]","[127, 127, 127, 127, 127]","[21000.0, 21000.0, 21000.0, 21000.0, 21000.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 20:42:58.900800
Offer_1002530118,"[0, 0, 0, 0]","[Application_1126987739, Application_112698773...","[0, 0, 0, 0]","[19750.0, 19750.0, 19750.0, 19750.0]","[210.12, 210.12, 210.12, 210.12]","[120, 120, 120, 120]","[20000.0, 20000.0, 20000.0, 20000.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 06:47:37.585000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,"[1, 1, 1, 1]","[Application_1786368292, Application_178636829...","[0, 0, 0, 0]","[1500.0, 1500.0, 1500.0, 1500.0]","[98.86, 98.86, 98.86, 98.86]","[66, 66, 66, 66]","[5500.0, 5500.0, 5500.0, 5500.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...",...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0 days 12:14:52.968750
Offer_993800442,"[1, 1, 1, 1]","[Application_1082798467, Application_108279846...","[0, 0, 0, 0]","[13000.0, 13000.0, 13000.0, 13000.0]","[392.88, 392.88, 392.88, 392.88]","[36, 36, 36, 36]","[13000.0, 13000.0, 13000.0, 13000.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 10:42:01.801250
Offer_99473283,"[0, 0, 0, 0, 0]","[Application_1183967721, Application_118396772...","[817, 817, 817, 817, 817]","[5500.0, 5500.0, 5500.0, 5500.0, 5500.0]","[98.86, 98.86, 98.86, 98.86, 98.86]","[66, 66, 66, 66, 66]","[5500.0, 5500.0, 5500.0, 5500.0, 5500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6 days 03:10:21.290600


In [121]:
def reduce_list_columns(df):
    new_df = a.copy()
    for col in df.columns:
        if(type(x) == list):
            control=0
            for i in range(len(df)):
                x = df[col].iloc[i]
                if len(set(x)) != 1:
                    control = 1
            if control ==0:
                new_df[col] = a[col].apply(lambda x: x[0])
    return new_df

In [123]:
b = reduce_list_columns(a)
b


,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_26,User_23,User_73,User_125,User_86,User_142,User_76,User_141,User_32,avg_t
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,1,Application_2131314372,956,14500.0,200.00,96,16000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 11:21:59.362200
Offer_1001553250,0,Application_1607028451,0,2500.0,175.00,127,17500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2 days 22:47:28.444400
Offer_1002136393,1,Application_638926349,969,6500.0,343.25,20,6500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5 days 22:58:29.630800
Offer_1002236598,0,Application_818693455,0,2500.0,210.00,127,21000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 20:42:58.900800
Offer_1002530118,0,Application_1126987739,0,19750.0,210.12,120,20000.0,0,"[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 06:47:37.585000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,1,Application_1786368292,0,1500.0,98.86,66,5500.0,0,"[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...",...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0 days 12:14:52.968750
Offer_993800442,1,Application_1082798467,0,13000.0,392.88,36,13000.0,0,"[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 10:42:01.801250
Offer_99473283,0,Application_1183967721,817,5500.0,98.86,66,5500.0,1,"[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6 days 03:10:21.290600


In [89]:
new_df = a.copy()
new_df['y'] = a['y'].apply(lambda x: x[0])
new_df['(case) CreditScore'] = a['(case) CreditScore'].apply(lambda x: x[0])
new_df['(case) FirstWithdrawalAmount'] = a['(case) FirstWithdrawalAmount'].apply(lambda x: x[0])
new_df['(case) MonthlyCost'] = a['(case) MonthlyCost'].apply(lambda x: x[0])
new_df['(case) NumberOfTerms'] = a['(case) NumberOfTerms'].apply(lambda x: x[0])
new_df['(case) OfferedAmount'] = a['(case) OfferedAmount'].apply(lambda x: x[0])
new_df['(case) Selected'] = a['(case) Selected'].apply(lambda x: x[0])

new_df

,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_26,User_23,User_73,User_125,User_86,User_142,User_76,User_141,User_32,avg_t
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,1,"[Application_2131314372, Application_213131437...",956,14500.0,200.00,96,16000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 11:21:59.362200
Offer_1001553250,0,"[Application_1607028451, Application_160702845...",0,2500.0,175.00,127,17500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2 days 22:47:28.444400
Offer_1002136393,1,"[Application_638926349, Application_638926349,...",969,6500.0,343.25,20,6500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5 days 22:58:29.630800
Offer_1002236598,0,"[Application_818693455, Application_818693455,...",0,2500.0,210.00,127,21000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1 days 20:42:58.900800
Offer_1002530118,0,"[Application_1126987739, Application_112698773...",0,19750.0,210.12,120,20000.0,0,"[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 06:47:37.585000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,1,"[Application_1786368292, Application_178636829...",0,1500.0,98.86,66,5500.0,0,"[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...",...,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0 days 12:14:52.968750
Offer_993800442,1,"[Application_1082798467, Application_108279846...",0,13000.0,392.88,36,13000.0,0,"[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,15 days 10:42:01.801250
Offer_99473283,0,"[Application_1183967721, Application_118396772...",817,5500.0,98.86,66,5500.0,1,"[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...",...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6 days 03:10:21.290600


In [ ]:
new_df

In [8]:
percent_missing = new_df.isnull().sum() * 100 / len(new_df)
missing_value_df = pd.DataFrame({'column_name': new_df.columns,
                                 'percent_missing': percent_missing})

# extract list of coumns which are more then 30%
miss_ls = list(percent_missing[percent_missing>30].keys())

# keep the remaining columns where missing data is less then 30%
trace = new_df.loc[:, ~new_df.columns.isin(miss_ls)]
trace.head()

,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_26,User_23,User_73,User_125,User_86,User_142,User_76,User_141,User_32,avg_t
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,1,"[Application_2131314372, Application_213131437...",956,14500.0,200.00,96,16000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 days 11:21:59.362200
Offer_1001553250,0,"[Application_1607028451, Application_160702845...",0,2500.0,175.00,127,17500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2 days 22:47:28.444400
Offer_1002136393,1,"[Application_638926349, Application_638926349,...",969,6500.0,343.25,20,6500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5 days 22:58:29.630800
Offer_1002236598,0,"[Application_818693455, Application_818693455,...",0,2500.0,210.00,127,21000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1 days 20:42:58.900800
Offer_1002530118,0,"[Application_1126987739, Application_112698773...",0,19750.0,210.12,120,20000.0,0,"[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15 days 06:47:37.585000


In [9]:
miss_ls

[]

In [10]:
# converting dataframe to event log
trace_log = pm4py.format_dataframe(df, case_id='caseid', activity_key='EventID', timestamp_key='ts')
trace_log = pm4py.convert_to_event_log(trace_log)

# add other time features
trace_log = interval_lifecycle.assign_lead_cycle_time(trace_log)

In [11]:
for i in range(len(df)):
    if df['caseid'][i] == 'Offer_1000681710':
        print(i)

14554
14555
14556
14557
14558


In [12]:
for i in range(len(df)):
    if df['caseid'][i] == 'Offer_1000681710':
        print("index is: " + str(i))
        print("EventID is:", end=" ")
        print(df['EventID'][i])

index is: 14554
EventID is: Offer_1000681710
index is: 14555
EventID is: OfferState_452902905
index is: 14556
EventID is: OfferState_256930279
index is: 14557
EventID is: OfferState_139676846
index is: 14558
EventID is: OfferState_983793285


trace_log: a list of lists of EventIDs for each caseid in df

In [13]:
traces_log_list = project_traces(trace_log)
traces_log_list

[['Offer_1000681710',
  'OfferState_452902905',
  'OfferState_256930279',
  'OfferState_139676846',
  'OfferState_983793285'],
 ['Offer_1001553250',
  'OfferState_1604351174',
  'OfferState_165310291',
  'OfferState_1913280206',
  'OfferState_1117047946'],
 ['Offer_1002136393',
  'OfferState_1608457034',
  'OfferState_1792705282',
  'OfferState_303325283',
  'OfferState_316331640'],
 ['Offer_1002236598',
  'OfferState_1780384890',
  'OfferState_2096359701',
  'OfferState_1571207942',
  'OfferState_756103056'],
 ['Offer_1002530118',
  'OfferState_1971804832',
  'OfferState_791185273',
  'OfferState_1078121548'],
 ['Offer_1002534091',
  'OfferState_1663136586',
  'OfferState_705687248',
  'OfferState_91701073'],
 ['Offer_1003919273',
  'OfferState_1789429921',
  'OfferState_2054863194',
  'OfferState_1467045452'],
 ['Offer_1004444125',
  'OfferState_1945050329',
  'OfferState_1157194888',
  'OfferState_61736097'],
 ['Offer_1004722883',
  'OfferState_978285621',
  'OfferState_491685430',


In [14]:
len(traces_log_list)

5000

In [15]:
df['caseid'].nunique()

5000

In [16]:
project_nth(trace_log, 4999)

['Offer_997411923', 'OfferState_1967587147', 'OfferState_600565321', 'OfferState_836197344', 'OfferState_892473715']


In [17]:
df.head()

,index,caseid,activity,resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,...,EventID,EventOrigin,OfferID,lifecycle:transition,t,year,month,case:concept:name,concept:name,time:timestamp
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,1,Application_196483749,0,...,Offer_247135719,Offer,Unknown,complete,0 days 00:00:03.042000,2016,1,Offer_247135719,Offer_247135719,2016-01-02 18:17:05.720
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,1,Application_196483749,0,...,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000,2016,1,Offer_247135719,OfferState_124849367,2016-01-02 18:17:08.762
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,1,Application_196483749,0,...,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000,2016,1,Offer_247135719,OfferState_440662877,2016-01-02 18:19:21.330
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,1,Application_196483749,0,...,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000,2016,1,Offer_247135719,OfferState_591416028,2016-01-02 18:21:26.034
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,1,Application_196483749,0,...,Offer_941964966,Offer,Unknown,complete,0 days 00:00:01.551000,2016,1,Offer_941964966,Offer_941964966,2016-01-02 18:21:42.022


In [18]:
df.shape

(22409, 26)

In [19]:
df.columns

Index(['index', 'caseid', 'activity', 'resource', 'ts', 'Variant',
       'Variant index', 'y', '(case) ApplicationID', '(case) CreditScore',
       '(case) FirstWithdrawalAmount', '(case) MonthlyCost',
       '(case) NumberOfTerms', '(case) OfferedAmount', '(case) Selected',
       'Action', 'EventID', 'EventOrigin', 'OfferID', 'lifecycle:transition',
       't', 'year', 'month', 'case:concept:name', 'concept:name',
       'time:timestamp'],
      dtype='object')

In [20]:
df_grouped= df.groupby(["caseid"]).agg(list)
df_grouped

,index,activity,resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,...,EventID,EventOrigin,OfferID,lifecycle:transition,t,year,month,case:concept:name,concept:name,time:timestamp
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,"[14554, 14555, 14556, 14557, 14558]","[O_Create Offer, O_Created, O_Sent (mail and o...","[User_20, User_20, User_20, User_117, User_115]","[2016-02-02 18:24:54.170000, 2016-02-02 18:24:...","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[1, 1, 1, 1, 1]","[Application_2131314372, Application_213131437...","[956, 956, 956, 956, 956]","[14500.0, 14500.0, 14500.0, 14500.0, 14500.0]",...,"[Offer_1000681710, OfferState_452902905, Offer...","[Offer, Offer, Offer, Offer, Offer]","[Unknown, Offer_1000681710, Offer_1000681710, ...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.663000, 0 days 00:00:19.05600...","[2016, 2016, 2016, 2016, 2016]","[2, 2, 2, 2, 2]","[Offer_1000681710, Offer_1000681710, Offer_100...","[Offer_1000681710, OfferState_452902905, Offer...","[2016-02-02 18:24:54.170000, 2016-02-02 18:24:..."
Offer_1001553250,"[5452, 5453, 5454, 5455, 5456]","[O_Create Offer, O_Created, O_Sent (mail and o...","[User_2, User_2, User_2, User_113, User_30]","[2016-01-16 03:16:41.868000, 2016-01-16 03:16:...","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[0, 0, 0, 0, 0]","[Application_1607028451, Application_160702845...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]",...,"[Offer_1001553250, OfferState_1604351174, Offe...","[Offer, Offer, Offer, Offer, Offer]","[Unknown, Offer_1001553250, Offer_1001553250, ...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.188000, 0 days 00:00:15.47200...","[2016, 2016, 2016, 2016, 2016]","[1, 1, 1, 1, 1]","[Offer_1001553250, Offer_1001553250, Offer_100...","[Offer_1001553250, OfferState_1604351174, Offe...","[2016-01-16 03:16:41.868000, 2016-01-16 03:16:..."
Offer_1002136393,"[11086, 11087, 11088, 11089, 11090]","[O_Create Offer, O_Created, O_Sent (mail and o...","[User_85, User_85, User_85, User_117, User_118]","[2016-01-29 17:46:49.529000, 2016-01-29 17:46:...","[Variant 2, Variant 2, Variant 2, Variant 2, V...","[2, 2, 2, 2, 2]","[1, 1, 1, 1, 1]","[Application_638926349, Application_638926349,...","[969, 969, 969, 969, 969]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]",...,"[Offer_1002136393, OfferState_1608457034, Offe...","[Offer, Offer, Offer, Offer, Offer]","[Unknown, Offer_1002136393, Offer_1002136393, ...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.293000, 0 days 00:00:19.55600...","[2016, 2016, 2016, 2016, 2016]","[1, 1, 1, 2, 2]","[Offer_1002136393, Offer_1002136393, Offer_100...","[Offer_1002136393, OfferState_1608457034, Offe...","[2016-01-29 17:46:49.529000, 2016-01-29 17:46:..."
Offer_1002236598,"[3518, 3519, 3520, 3521, 3522]","[O_Create Offer, O_Created, O_Sent (mail and o...","[User_49, User_49, User_49, User_113, User_102]","[2016-01-12 18:24:55.557000, 2016-01-12 18:24:...","[Variant 3, Variant 3, Variant 3, Variant 3, V...","[3, 3, 3, 3, 3]","[0, 0, 0, 0, 0]","[Application_818693455, Application_818693455,...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]",...,"[Offer_1002236598, OfferState_1780384890, Offe...","[Offer, Offer, Offer, Offer, Offer]","[Unknown, Offer_1002236598, Offer_1002236598, ...","[complete, complete, complete, complete, compl...","[0 days 00:00:01.269000, 0 days 00:00:11.34800...","[2016, 2016, 2016, 2016, 2016]","[1, 1, 1, 1, 1]","[Offer_1002236598, Offer_1002236598, Offer_100...","[Offer_1002236598, OfferState_1780384890, Offe...","[2016-01-12 18:24:55.557000, 2016-01-12 18:24:..."
Offer_1002530118,"[21512, 21513, 21514, 21515]","[O_Create Offer, O_Created, O_Sent (mail and o...","[User_15, User_15, User_15, User_1]","[2016-02-17 02:25:07.160000, 2016-02-17 02:25:...","[Variant 1, Variant 1, Variant 1, Variant 1]","[1, 1, 1, 1]","[0, 0, 

In [25]:
final_df  = my_utils.index_encoding(df)
final_df

,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_23_5,User_73_5,User_125_5,User_86_5,User_142_5,User_76_5,User_141_5,User_32_5,t_5,out
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,"[1, 1, 1, 1, 1]","[Application_2131314372, Application_213131437...","[956, 956, 956, 956, 956]","[14500.0, 14500.0, 14500.0, 14500.0, 14500.0]","[200.0, 200.0, 200.0, 200.0, 200.0]","[96, 96, 96, 96, 96]","[16000.0, 16000.0, 16000.0, 16000.0, 16000.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0,0,0,0,0,0,0,0,2016-02-10 00:55:41.368000,1
Offer_1001553250,"[0, 0, 0, 0, 0]","[Application_1607028451, Application_160702845...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[175.0, 175.0, 175.0, 175.0, 175.0]","[127, 127, 127, 127, 127]","[17500.0, 17500.0, 17500.0, 17500.0, 17500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0,0,0,0,0,0,0,0,2016-01-26 19:14:10.052000,0
Offer_1002136393,"[1, 1, 1, 1, 1]","[Application_638926349, Application_638926349,...","[969, 969, 969, 969, 969]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[343.25, 343.25, 343.25, 343.25, 343.25]","[20, 20, 20, 20, 20]","[6500.0, 6500.0, 6500.0, 6500.0, 6500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0,0,0,0,0,0,0,0,2016-02-22 15:42:42.278000,1
Offer_1002236598,"[0, 0, 0, 0, 0]","[Application_818693455, Application_818693455,...","[0, 0, 0, 0, 0]","[2500.0, 2500.0, 2500.0, 2500.0, 2500.0]","[210.0, 210.0, 210.0, 210.0, 210.0]","[127, 127, 127, 127, 127]","[21000.0, 21000.0, 21000.0, 21000.0, 21000.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0,0,0,0,0,0,0,0,2016-01-18 22:33:14.113000,0
Offer_1002530118,"[0, 0, 0, 0]","[Application_1126987739, Application_112698773...","[0, 0, 0, 0]","[19750.0, 19750.0, 19750.0, 19750.0]","[210.12, 210.12, 210.12, 210.12]","[120, 120, 120, 120]","[20000.0, 20000.0, 20000.0, 20000.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,"[1, 1, 1, 1]","[Application_1786368292, Application_178636829...","[0, 0, 0, 0]","[1500.0, 1500.0, 1500.0, 1500.0]","[98.86, 98.86, 98.86, 98.86]","[66, 66, 66, 66]","[5500.0, 5500.0, 5500.0, 5500.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...",...,0,1,0,0,0,0,0,0,0,1
Offer_993800442,"[1, 1, 1, 1]","[Application_1082798467, Application_108279846...","[0, 0, 0, 0]","[13000.0, 13000.0, 13000.0, 13000.0]","[392.88, 392.88, 392.88, 392.88]","[36, 36, 36, 36]","[13000.0, 13000.0, 13000.0, 13000.0]","[0, 0, 0, 0]","[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...",...,0,0,0,0,0,0,0,0,0,1
Offer_99473283,"[0, 0, 0, 0, 0]","[Application_1183967721, Application_118396772...","[817, 817, 817, 817, 817]","[5500.0, 5500.0, 5500.0, 5500.0, 5500.0]","[98.86, 98.86, 98.86, 98.86, 98.86]","[66, 66, 66, 66, 66]","[5500.0, 5500.0, 5500.0, 5500.0, 5500.0]","[1, 1, 1, 1, 1]","[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...",...,0,0,0,0,0,0,0,0,2016-02-22 16:13:10.784000,0


In [26]:
new_df = final_df.copy()
new_df['y'] = final_df['y'].apply(lambda x: x[0])
new_df['(case) CreditScore'] = final_df['(case) CreditScore'].apply(lambda x: x[0])
new_df['(case) FirstWithdrawalAmount'] = final_df['(case) FirstWithdrawalAmount'].apply(lambda x: x[0])
new_df['(case) MonthlyCost'] = final_df['(case) MonthlyCost'].apply(lambda x: x[0])
new_df['(case) NumberOfTerms'] = final_df['(case) NumberOfTerms'].apply(lambda x: x[0])
new_df['(case) OfferedAmount'] = final_df['(case) OfferedAmount'].apply(lambda x: x[0])
new_df['(case) Selected'] = final_df['(case) Selected'].apply(lambda x: x[0])

new_df

,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,(case) NumberOfTerms,(case) OfferedAmount,(case) Selected,Action,EventID,...,User_23_5,User_73_5,User_125_5,User_86_5,User_142_5,User_76_5,User_141_5,User_32_5,t_5,out
caseid,,,,,,,,,,,,,,,,,,,,,
Offer_1000681710,1,"[Application_2131314372, Application_213131437...",956,14500.0,200.00,96,16000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1000681710, OfferState_452902905, Offer...",...,0,0,0,0,0,0,0,0,2016-02-10 00:55:41.368000,1
Offer_1001553250,0,"[Application_1607028451, Application_160702845...",0,2500.0,175.00,127,17500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1001553250, OfferState_1604351174, Offe...",...,0,0,0,0,0,0,0,0,2016-01-26 19:14:10.052000,0
Offer_1002136393,1,"[Application_638926349, Application_638926349,...",969,6500.0,343.25,20,6500.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002136393, OfferState_1608457034, Offe...",...,0,0,0,0,0,0,0,0,2016-02-22 15:42:42.278000,1
Offer_1002236598,0,"[Application_818693455, Application_818693455,...",0,2500.0,210.00,127,21000.0,1,"[Created, statechange, statechange, statechang...","[Offer_1002236598, OfferState_1780384890, Offe...",...,0,0,0,0,0,0,0,0,2016-01-18 22:33:14.113000,0
Offer_1002530118,0,"[Application_1126987739, Application_112698773...",0,19750.0,210.12,120,20000.0,0,"[Created, statechange, statechange, statechange]","[Offer_1002530118, OfferState_1971804832, Offe...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Offer_993689039,1,"[Application_1786368292, Application_178636829...",0,1500.0,98.86,66,5500.0,0,"[Created, statechange, statechange, statechange]","[Offer_993689039, OfferState_1192570219, Offer...",...,0,1,0,0,0,0,0,0,0,1
Offer_993800442,1,"[Application_1082798467, Application_108279846...",0,13000.0,392.88,36,13000.0,0,"[Created, statechange, statechange, statechange]","[Offer_993800442, OfferState_1754231388, Offer...",...,0,0,0,0,0,0,0,0,0,1
Offer_99473283,0,"[Application_1183967721, Application_118396772...",817,5500.0,98.86,66,5500.0,1,"[Created, statechange, statechange, statechang...","[Offer_99473283, OfferState_1945701229, OfferS...",...,0,0,0,0,0,0,0,0,2016-02-22 16:13:10.784000,0


# Data encoding

    - boolean encoding (one hot encoding if the activity present or not)
    - frequency encoding (count of activity)
    - simple index encoding (n events one hot encoding)


In [28]:
# function to get the one hot encoded vectors of categorical values
def get_ohe_dict(categorical_vars, df):
    ohe_dict = {}
    for var in categorical_vars:
        var_dict = {}
        var_data = sorted(df[var].unique())
        var_len = len(var_data)
        for i,cat in enumerate(var_data):
            var_dict[cat] = [0]*var_len
            var_dict[cat][i] = 1

        ohe_dict[var] = var_dict

    return ohe_dict

In [29]:
# padding function for ohe encoding
def cat_padding(vec, t_length, attr_length):
    desired_length = t_length*attr_length
    vec_length = len(vec)
    if vec_length != desired_length:
        pad_vec = [0]*(desired_length-vec_length)
        vec.extend(pad_vec)
    return vec

In [30]:
# padding function for non-ohe encoding
def num_padding(vec, t_length):
    vec_length = len(vec)
    if vec_length != t_length:
        pad_vec = [0]*(t_length-vec_length)
        vec.extend(pad_vec)
    return vec

In [43]:
df_encoded = pd.get_dummies(df, columns=['activity'])
df_encoded = pd.get_dummies(df_encoded, columns=['Variant'])
df_encoded = pd.get_dummies(df_encoded, columns=['Action'])
df_encoded = pd.get_dummies(df_encoded, columns=['EventOrigin'])
df_encoded = pd.get_dummies(df_encoded, columns=['lifecycle:transition'])
df_encoded

,index,caseid,resource,ts,Variant index,y,(case) ApplicationID,(case) CreditScore,(case) FirstWithdrawalAmount,(case) MonthlyCost,...,Variant_Variant 3,Variant_Variant 4,Variant_Variant 5,Variant_Variant 6,Variant_Variant 7,Variant_Variant 8,Action_Created,Action_statechange,EventOrigin_Offer,lifecycle:transition_complete
0,0,Offer_247135719,User_17,2016-01-02 18:17:05.720,8,1,Application_196483749,0,10000.0,201.76,...,0,0,0,0,0,1,1,0,1,1
1,1,Offer_247135719,User_17,2016-01-02 18:17:08.762,8,1,Application_196483749,0,10000.0,201.76,...,0,0,0,0,0,1,0,1,1,1
2,2,Offer_247135719,User_17,2016-01-02 18:19:21.330,8,1,Application_196483749,0,10000.0,201.76,...,0,0,0,0,0,1,0,1,1,1
3,3,Offer_247135719,User_17,2016-01-02 18:21:26.034,8,1,Application_196483749,0,10000.0,201.76,...,0,0,0,0,0,1,0,1,1,1
4,4,Offer_941964966,User_17,2016-01-02 18:21:42.022,1,1,Application_196483749,0,4100.0,201.76,...,0,0,0,0,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,User_23,2016-02-18 22:50:01.446,2,1,Application_1574359134,767,6000.0,118.56,...,0,0,0,0,0,0,1,0,1,1
22405,22405,Offer_1064426652,User_23,2016-02-18 22:50:02.655,2,1,Application_1574359134,767,6000.0,118.56,...,0,0,0,0,0,0,0,1,1,1
22406,22406,Offer_1064426652,User_23,2016-02-18 22:50:22.174,2,1,Application_1574359134,767,6000.0,118.56,...,0,0,0,0,0,0,0,1,1,1
22407,22407,Offer_1064426652,User_117,2016-02-29 19:08:56.200,2,1,Application_1574359134,767,6000.0,118.56,...,0,0,0,0,0,0,0,1,1,1


In [ ]:
df_encoded[]

In [39]:
str_ev_attr = ['activity', 'EventOrigin','lifecycle:transition']

In [35]:
df

,index,caseid,activity,resource,ts,Variant,Variant index,y,(case) ApplicationID,(case) CreditScore,...,EventID,EventOrigin,OfferID,lifecycle:transition,t,year,month,case:concept:name,concept:name,time:timestamp
0,0,Offer_247135719,O_Create Offer,User_17,2016-01-02 18:17:05.720,Variant 8,8,1,Application_196483749,0,...,Offer_247135719,Offer,Unknown,complete,0 days 00:00:03.042000,2016,1,Offer_247135719,Offer_247135719,2016-01-02 18:17:05.720
1,1,Offer_247135719,O_Created,User_17,2016-01-02 18:17:08.762,Variant 8,8,1,Application_196483749,0,...,OfferState_124849367,Offer,Offer_247135719,complete,0 days 00:02:12.568000,2016,1,Offer_247135719,OfferState_124849367,2016-01-02 18:17:08.762
2,2,Offer_247135719,O_Sent (online only),User_17,2016-01-02 18:19:21.330,Variant 8,8,1,Application_196483749,0,...,OfferState_440662877,Offer,Offer_247135719,complete,0 days 00:02:04.704000,2016,1,Offer_247135719,OfferState_440662877,2016-01-02 18:19:21.330
3,3,Offer_247135719,O_Cancelled,User_17,2016-01-02 18:21:26.034,Variant 8,8,1,Application_196483749,0,...,OfferState_591416028,Offer,Offer_247135719,complete,0 days 00:02:04.704000,2016,1,Offer_247135719,OfferState_591416028,2016-01-02 18:21:26.034
4,4,Offer_941964966,O_Create Offer,User_17,2016-01-02 18:21:42.022,Variant 1,1,1,Application_196483749,0,...,Offer_941964966,Offer,Unknown,complete,0 days 00:00:01.551000,2016,1,Offer_941964966,Offer_941964966,2016-01-02 18:21:42.022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22404,22404,Offer_1064426652,O_Create Offer,User_23,2016-02-18 22:50:01.446,Variant 2,2,1,Application_1574359134,767,...,Offer_1064426652,Offer,Unknown,complete,0 days 00:00:01.209000,2016,2,Offer_1064426652,Offer_1064426652,2016-02-18 22:50:01.446
22405,22405,Offer_1064426652,O_Created,User_23,2016-02-18 22:50:02.655,Variant 2,2,1,Application_1574359134,767,...,OfferState_189916093,Offer,Offer_1064426652,complete,0 days 00:00:19.519000,2016,2,Offer_1064426652,OfferState_189916093,2016-02-18 22:50:02.655
22406,22406,Offer_1064426652,O_Sent (mail and online),User_23,2016-02-18 22:50:22.174,Variant 2,2,1,Application_1574359134,767,...,OfferState_24747308,Offer,Offer_1064426652,complete,10 days 20:18:34.026000,2016,2,Offer_1064426652,OfferState_24747308,2016-02-18 22:50:22.174
22407,22407,Offer_1064426652,O_Returned,User_117,2016-02-29 19:08:56.200,Variant 2,2,1,Application_1574359134,767,...,OfferState_1809485991,Offer,Offer_1064426652,complete,0 days 04:34:49.631000,2016,2,Offer_1064426652,OfferState_1809485991,2016-02-29 19:08:56.200


In [36]:
categorical_vars = ['activity', 'EventOrigin','lifecycle:transition']
ohe_dict = get_ohe_dict(categorical_vars, df)

In [40]:
# here for each trace we extract ohe vector for activity and sum them up 
# because this encoding only provides count of how many times the activity appears

data = []

for id, group in df.groupby(['caseid']):
    feature_vec = []

    # add categorical and numerical event attributes
    for cat_atr in str_ev_attr[:1]:
        attr_length = len(list(ohe_dict[cat_atr].values())[0])
        str_ev_vec = np.array([0]*attr_length)

        for ca in group[cat_atr]:
            str_ev_vec  = str_ev_vec + np.array(ohe_dict[cat_atr][ca])

        feature_vec.extend(list(str_ev_vec))

    data.append(feature_vec)

data

[[1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 0],
 